# Online Korean Community Forum Post Generator

## CS344 Final Report

05/21/2020

*Ian Park*

## Vision
Ever since I watched Microsoft's Twitter bot, [Tay](https://en.wikipedia.org/wiki/Tay_(bot)), I was intrigued by AIs that could seamlessly imitate Human communication. Although Tay was somewhat of a disaster, it was able to generate text that embodied the personality or flair of the crowd it was interacting with. In Tay's case, this was the twitter community trying to make a laugh out of it. Similarly, each online community forms its own way of speaking. I was curious if models could learn these unique styles of speaking. For this project, I wanted to do exactly that. I wanted to create a bot that generated text that imitated certain online communities.

Because I wanted to capture the style of conversation on these online communities through a model, I needed a comprehensive platform to easily do that. So, part of the project was to utilize all the tools one would need to create an ML-assisted bot given an online community. This comprises a web scraper (Scrapy), the machine learning model (Keras), and a proof-of-concept application with Flask.

The final goal with my project was to explore Korean-speaking communities rather than English ones because there were already many existing bots.

## Background
### Scraping
Since the purpose of the project was to capture the style of certain online communities, I could not use any off-the-shelf word datasets for my project. Because of my third goal, even if I wanted to, I would not have been able to find a decent community forum post data. If I was to train my data with English, I would've gone with some data on Github like this: [linanqiu/reddit-dataset](https://github.com/linanqiu/reddit-dataset).
Nevertheless, because there wasn't any data promptly available, I had to create my own data set. Fortunately, these internet communities did not lack any data. There were at least thousands of posts every day.
Furthermore, there was an amazing web-scraping framework in python called [Scrapy](https://scrapy.org/). This allowed one to create website scrapers/crawlers by creating what they call a "spider". A spider is a set of instructions for scraping a particular website. I've created two unique spiders for this project, that is, to scrape two sites, [DCInside Comics](https://gall.dcinside.com/board/lists?id=comic_new2) and [Talk OP.GG](https://talk.op.gg/s/lol/). Through Scrapy you can automate all your spider instances to get the data and format it as well.

### Choosing a Model: A Comparison Natural Language Generation Approaches
***Markov Chain.*** There are multiple ways that large tech companies like Microsoft or Google generate seamless natural text. On the other hand, there is also a way to achieve this with Hidden Markov Chains without neural networks. As we've seen when implementing a variation in CS212, although fast, these are not so convincing. It turns out that they are good enough in the short term with smaller data but not so good compared to neural networks in the long term ([Panzner, M. Cimiano, P](https://pub.uni-bielefeld.de/download/2903474/2907910)) given larger data. In hindsight, given the amount of data that I had, this could've been a viable option for this project.

***Transformers.*** There is even a huge collection of general, modern, state-of-the-art NLG (Natural Language Generation) models that even do the pre-training for you [huggingfaces/transformers](https://github.com/huggingface/transformers). These implementations are built on a completely different architecture called transformers, which eliminate recurrence and convolution to replace them with personal attention to establish dependencies between inputs and outputs ([Benaffane](https://medium.com/analytics-vidhya/transformer-vs-rnn-and-cnn-18eeefa3602b)). They seem to give the most realistic results, hence most widely used in recent systems.

The most prominent transformer model seemed to be GPT-2, which was released by [OpenAI](https://openai.com/blog/better-language-models/).

The common practice of using Transformer architectures is to be to "fine-tune" the trained model to another set of words ([OpenAI](https://openai.com/blog/fine-tuning-gpt-2/)). If one is generating text in English, there was a really nice off-the-shelf library called [huggingface transformers](https://github.com/huggingface/transformers), which included a pre-trained model for general automatic text-generation.

Similarly, there was a project for Korean called KoGPT2 by a company called SK, which is the third-largest company in Korea. They've trained their own model using Wiki data and online news data ([SKT-AI/KoGPT2](https://github.com/SKT-AI/KoGPT2)).

***RNN with LSTM*** Recurrent Neural Networks is powerful for modeling sequence data such as time series or natural language. Unlike Vanilla NNs and CNNs, RNNs combine the input vector with their state vector with a learned function to produce a new state vector. This gives RNNs the power to process sequential data well. However, traditional RNN or "vanilla" RNN suffers from the vanishing gradient, in which the model cannot remember too far back in time ([Sherstinsky](https://arxiv.org/pdf/1808.03314.pdf)).

Fortunately, LSTM (Long Short Term Memory) networks can solve this by having cells and gates to emulate memory over arbitrary time intervals. This allows the model to remember what happened many time-steps ago. Overall, RNNs can learn how to generate languages because languages are sequential. This was the primary model I trained my data on.

I heavily referenced Chollet's [notebook](https://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb) on LSTMs for the implementation. And conceptually referenced, [Karpathy](https://karpathy.github.io/2015/05/21/rnn-effectiveness/), to understand how RNNs work.

### The Application
The application I made was to show how one could make an ML model serviceable. There is nothing too fancy here. I created a [Flask](https://flask.palletsprojects.com/) REST API that would return the generated text of a selected community and display it on a website.

At first, I was tempted to extend this service to automate the scraping and the training process as well. However, unless one was doing this for commercial purposes, it seemed unfeasible to train your model on the cloud. Even with cheaper [AWS options](https://aws.amazon.com/getting-started/projects/build-machine-learning-model/services-costs/), the cost was hourly.
Furthermore, it was out of the scope of this class.  So, I just opted to train my model separately on a paid Google Colab instance.


## The Implementation
### The Scraper
***Story*** Before I go into the specific detail, I want to say that one should not attempt to do an extensive scraping to collect machine learning data without doing some background research. Although I've done scraping in the past, I never tried scraping more than 50,000 pages in such a short period. So, my initial attempts at scraping for this project got both my IP address and account banned from a [site](https://bbs.ruliweb.com/community). Likewise, even if a website does not ban, one should approach scraping hospitably. There are several precautions that [Scrapy recommends](https://docs.scrapy.org/en/latest/topics/practices.html). Furthermore, I will outline some things I implemented to avoid getting banned.

***Implementation*** A Scrapy [spider](https://docs.scrapy.org/en/latest/topics/spiders.html) is a crawler for a specifically targeted website. You can handle custom arguments and have per-spider settings for customization. The primary part of creating a spider is parsing the HTML from the website. This process probably took the third-most time and probably the most implementation time (training and scraping took the longest).

***Avoiding getting banned*** These are some good practices I took to avoid getting banned.
1. *Rotate your IP address using a proxy or TOR*. In python, there is a module called `toripchanger` that allows you to easily rotate new tor instances using `privoxy` and tor. Here's a [detailed setup guide](https://gist.github.com/DusanMadar/8d11026b7ce0bce6a67f7dd87b999f6b.) for Linux systems 
2. *Rotate your user-agent.* Each browser or web client has a user-agent based on its browser type, version, and HTML engine. For instance, `'Mozilla/5.0 (X11; Linux x86_64) ' 'AppleWebKit/537.36 (KHTML, like Gecko) ' 'Chrome/62.0.3202.89 ' 'Safari/537.36'`. Rotating and spoofing the scraper client's user agent is simple. In Scrapy, this is easily done using the [`UserAgentsMiddleware`](https://pypi.org/project/scrapy-user-agents/).
3. *Set a delay*. This is probably the primary cause that would get you banned from a site. If you run your scraper unrestricted for many hours, the server will notice unusual spikes and possibly cut you off to maintain a better experience for other users. In Scrapy you can easily set a delay.

### Keras LSTM
***Data Cleanup*** Due to the nature of the dataset (i.e. informal and slang-heavy), this part was crucial to filter out a lot of over-fitting and random behavior. For instance, one of the communities I scraped from was a discussion board on Anime. So, the conversations included Korean, English, Japanese, Chinese characters, and Emojis. I just range-eliminated most non-Korean and non-English text including punctuations. Punctuations were also causing a lot of problems because there would be many '!!!!!!!', '?????', or '.........' to express exaggeration. I also had a problem with Korean characters but I will discuss them with more detail in the results section.

***Word Embeddings or One-Hot-Encoding?*** I did not choose to utilize a word embedding layer instead of a dumb character vector because information on Korea was limited and I wasn't sure if it would capture weird ungrammatical words and pseudowords as well. So, I went with plain-old one-hot-encoding

***Sequences*** For the character-based network to work properly, I needed to divide the corpus into sequences of equal length. I decided to go with a sequence length of 20 and a sequence step of 2 characters. After testing it with various steps and sequence lengths, I found that changing it did not improve the results. Although the the official [Keras documentation](https://keras.io/examples/lstm_text_generation/)) suggests a length of 40, the average length of my the posts in my dataset were shorter.

***Initial Model*** In my initial model, I heavily referenced Chollet's implementation in the notebook on LSTMs. So, I added an LSTM layer with 128 units. The shape of the input was a matrix of `max_length` characters, in which each character is a vector of the length of total characters. I also added a fully-connected dense layer with softmax activation to pick the character the highest probability. The optimizer was the `categorical_crossentropy` loss function, which is for multi-class classification.

**Overfitting In Initial Model**:
![image1](img/overfitting-1.png)
![image1](img/overfitting-2.png)


***Finalized Model*** From my previous model, I made some adjustments so that it would fix some problems I was having with the simpler model. First, the previous model was over-fitting a lot even after 300 epochs. So I added another LSTM layer and a 20% dropout on each LSTM layer. Likewise, I added some nice callbacks to checkpoint/save the best model and to print a sample of a model.

***Training*** I was able to train this particular model for about 1200 epochs and get a loss .

### The Application
The application is my primary demo proof of concept of my model. It uses Flask with all of its niceties: templating, routing, etc. Writing the API was a bliss. Honestly, it took more time to write the nice HTML frontend than to make the server communicate the APIs with the web page.


## Results and Conclusion
### The Scraper
I was able to scrape about 2MB for each site over several days. Although this process would've sped up if I were to risk getting banned, I did not do so after getting banned on my first attempt. Nevertheless, I cannot conclude that this was sufficient data to train a model to speak Korean. However, in the scope of this project, it served me well. Also, I will be able to scrape more data in the future.

### The Application
As explained above, it is really basic right now to evaluate its result in terms of efficiency/scalability, but for this task, it works well.

### Model
Overall my model was able to generate at least some meaningful word sequences are given certain conditions I will explain below.

#### Positive Results 
First, the positive result was that my model was able to generate structured Korean text. Although there was not much meaning. Random repetition due to over-fitting did not appear. It most cases it also correctly put [postposition articles](https://en.wikipedia.org/wiki/Korean_postpositions) in most words. Also, it filtered out a lot of pseudo text I will explain below.

#### Negative Results
***Dealing with Pseudo Characters*** The first thing to understand is that since this is a character-based LSTM model, the language didn't matter. Or so I thought. The biggest challenge that the model had was the sheer length of characters the Korean language had. In almost every example, in English, people were trying to slim down their text data by getting rid of extraneous characters and they complained that their text data had more characters than the English alphabet. I never was aware of this but the Korean language represented in English was very large. It took almost 1000 characters to represent all of the unique combinations of the Korean alphabet that were used in the dataset. Of course, I wasn't using all of it. In total, there are [11,172 possible permutations](https://en.wikipedia.org/wiki/Hangul_Syllables) of the Korean alphabet that can be represented as a character in UTF-8. Even as a native Korean speaker, I never realized this until now. For use out of the computer, the language is pretty easy to learn as it is a phonetic language rather than an ideograph language like Chinese. When written by hand the alphabet system itself is very concise and elegant. Likewise, there aren't many situations where you need all of these permutations. Online Korean users, utilize this aspect of Korean to generate various pseudo-writings to convey meaning. For instance, one could represent, 어디가 as 없딦값. If you look closely, above the ㅂㅅ of each character of the latter word, you can see the characters 어,디, and 가. But, 딦 is never used anywhere else in Korean. 없and값are also used rarely. Something similar in English would be LeetSpeak in which only proficient English users and people who are familiar with the representation of the words can barely recognize them. Because playful nature of the online communities, these weird pseudo characters flooded my dataset and it was difficult to programmatically remove them as some characters that could seem bizarre in one sequence could have a purpose in another sequence.

However, one of the significant improvements in my model was filtering out obscure pseudo characters. In my final model, it had almost no sequences that contained these models. However, it was not able to catch their unique meanings but merely filtered it out. I was not seeing them as often, but I also never saw them in a meaningful (to the people who use it) way. I don't think that more training would not help. However, given more data, the model could improve to capture actual pseudo characters according to their meanings.

**Pseudo Characters:**
![image1](img/pseudochars-1.png)
![image1](img/pseudochars-2.png)
![image1](img/pseudochars-3.png)


***Korean word endings*** Another unusual characteristic of Korean is that specific post-positional characters are reserved for sentence endings. This is true for honorifics and also slang endings. I compiled a "blacklist" of these unique "ending" characters. 어,네,함,감,듬,슴,지,삼,듯,음,야,럼,냐,내,다,임,야,옴,한다,주라,요,겠, 구나,네. Unlike English which delineated sentence structure by words rather than characters, Korean do this by character. Thus, given a corpus, it is easier to for a machine learning the actual language (by sequence) after learning the character. Because the dataset primarily consisted of single posts that usually ended with one sentence, there was not enough data for my model to learn how to respond to these endings characters when seeded with them.

**Bad Word Endings** (There is not much structural spacing and grammatical propositions are ignored):
![image1](img/lstm-1.png)
![image1](img/lstm-2.png)
![image1](img/lstm-3.png)

**Good Word Endings** (You can notice by the model respecting grammatical postpositions):
![image1](img/lstm-good-1.png)
![image1](img/lstm-good-2.png)
![image1](img/lstm-good-3.png)


#### Hacky Solution

This problem was somewhat solved by selectively choosing the seed for the predictions. As I said, I "blacklisted" the bad ending characters. Although this is kind of a hack, it improved the results significantly.

### Conclusion
Overall, I think the project worked well in the scope of this class. I could've benefited from thinking about it and scraping more data earlier on in the semester. I also learned that the Korean language is very computer-unfriendly and a simple character-based LSTM might not be the best solution to generating text in Korean.

## Extra (For Fun!)
### Trying out GPT-2 and PyTorch
I will not explicitly detail the process of writing this code as I did with my RNN as this was more of me toying around with it. For the code, look at `gpt2.ipynb` it heavily references code from [KoGPT2-FineTuning](https://github.com/gyunggyung/KoGPT2-FineTuning/). Likewise, I did not do extensive conceptual research on transformer architectures. In fact, initially, I did not even consider using this architecture at all. However, during my frustration with the LSTM, I played around with it to see some interesting results. 

First, I thought that fine-tuning was an easier process than actual training but quickly realized that fine-tuning *was* training, and it took much longer to train than my simple model with two LSTM layers. Furthermore, I was able to fine-tune the KoGPT2 model for 30 epochs before the RAM on Google Colab (25GB on the Pro version) ran out and my personal Google Drive space also ran out because of the sheer size of the compiled weights. I did not upload the weights or provide them through a link just due to the impracticality of importing it every time to demonstrate it.

The results were disappointing. First, because of the aforementioned memory failures, I wasn't able to complete the fine-tuning with my dataset. Second, the pre-trained data influenced and override my data too strongly. Although it was much better than my model at formulating actually meaningful phrases, every prediction had a formal nuance rather than something that someone from the community I scraped the data from would say. This was most apparent, again, with ending characters. In Korean ending characters determine honorifics. Because the wiki and online news data that KoGPT2 was trained on were mostly formal data that contained strict honorifics, the fine-tuning with merely 30 epochs was far from sufficient. I guess more data and better hardware would solve the problem.

Here are some examples:
![image1](img/gpt-3.png)
![image1](img/gpt-2.png)
![image1](img/gpt-1.png)

### Future Goals
I want to continue to work on the project on a separate repository. Some goals that I have right now are to integrate the scraping and training into the backend. By doing so, I could automate scraping and training in intervals with maybe a cronjob to schedule sparse sessions of scraping and/or training. Likewise, I want to revise my model to be better at handling text by adding more layers. Furthermore, it would be nice to automate posting to the sites and respond to comments. Also, I hope to explore with different models. Perhaps I could add a feedback mechanism like Microsoft's Tay had.


## Bibliography

OpenAI (2019). "Fine-Tuning GPT-2 from Human Preferences." Accessed on May 19, 2020. https://openai.com/blog/fine-tuning-gpt-2/.

OpenAI (2019). "Better Language Models and Their Implications." Accessed on May 19, 2020. https://openai.com/blog/better-language-models/.

Chollet, F. "Text Generation with LSTM", Accessed on May 19, 2020. https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb.

Karpathy, A (2015). "The Unreasonable Effectiveness of Recurrent Neural Networks." Accessed on May 19, 2020. https://karpathy.github.io/2015/05/21/rnn-effectiveness.

Benaffane, Y (2019). "Transformer vs RNN and CNN for Translation Task", Accessed May 19, 2020. https://medium.com/analytics-vidhya/transformer-vs-rnn-and-cnn-18eeefa3602b.

Sherstinsky, A (2020). "Fundamentals  of  Recurrent  Neural  Network (RNN) and Long Short-Term Memory (LSTM) Network". Accessed May 19, 2020. https://arxiv.org/pdf/1808.03314.pdf.

Panzner, M. Cimiano, P (2016). "Comparing Hidden Markov Models and Long Short Term Memory Neural Networks for Learning Action Representation". https://pub.uni-bielefeld.de/download/2903474/2907910.

Wikipedia (2020). "Tay (bot).", Accessed on May 19, 2020, last edited on May 19, 2020. https://en.wikipedia.org/wiki/Tay_(bot).

Wikipedia (2020). "Korean postpositions.", Accessed on May 19, 2020, last edited on February 6, 2020. https://en.wikipedia.org/wiki/Korean_postpositions.

Wikipedia (2020). "Hangul Syllables.", Accessed on May 19, 2020, last edited on August 21, 2019. https://en.wikipedia.org/wiki/Hangul_Syllables.

Talk OP.GG. 전체 게시판 - 리그오브레전드, Accessed on May 19, 2020. https://talk.op.gg/s/lol/.

DCInside Cartoons. “만화 갤러리.” 디시인사이드, Accessed on May 19, 2020. https://gall.dcinside.com/board/lists?id=comic_new2.

Ruliweb. "커뮤니티 | 루리웹". Accessed on May 19, 2020. https://bbs.ruliweb.com/community

Github. linaqiu/reddit-dataset, Accessed on May19, 2020. https://github.com/linanqiu/reddit-dataset.

Github. SKT-AI/KoGPT2, Accessed on May 19, 2020. https://github.com/SKT-AI/KoGPT2.

Github. gyunggyung/KoGPT2-FineTuning, Accessed on May 19, 2020. https://github.com/gyunggyung/KoGPT2-FineTuning.

Github. huggingface/transformers, Accessed on May 19, 2020. https://github.com/huggingface/transformers.

Github. DusanMadar/8d11026b7ce0bce6a67f7dd87b999f6b, Accessed on May 19, 2020. https://gist.github.com/DusanMadar/8d11026b7ce0bce6a67f7dd87b999f6b.

Flask Documentation. Accessed on May 19, 2020. https://flask.palletsprojects.com/en/1.1.x/.

Keras Documentation. "LSTM for text generation" Accessed on May 19, 2020. https://keras.io/examples/lstm_text_generation/.

Scrapy Documentation. "Spiders". Accessed on May 19, 2020, https://docs.scrapy.org/en/latest/topics/spiders.html.

Scrapy Documentation. "Common Practices". Accessed on May 19, 2020, https://docs.scrapy.org/en/latest/topics/practices.html.

Scrapy. Accessed on May 19, 2020. https://scrapy.org/.

PyPI. "scrapy-user-agents". Accessed on May 19, 2020. https://pypi.org/project/scrapy-user-agents/.

PyPI. "toripchanger". Accessed on May 19, 2020. https://pypi.org/project/toripchanger/.

Amazon. "Services and Costs." Accessed on May 19, 2020. https://aws.amazon.com/getting-started/projects/build-machine-learning-model/services-costs.